In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [0]:
catalog_dev = "`land_topografisk-gdb_dev`"
schema_dev = "ai2025"

spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")

poly_over_tbl = "polygons_status_overview"
us_over_tbl = "utensnuplass_status_overview"

df_all = spark.read.table(poly_over_tbl)
df_no = spark.read.table(us_over_tbl)

In [0]:
df_yes = df_all.join(
    df_no.select("row_hash"),
    on="row_hash",
    how="left_anti"
)
count_no = df_no.count()
count_yes = df_yes.count()

p_no = 0.01
total_desired = count_no + count_yes
desired_no = int(total_desired * p_no)
desired_yes = total_desired - desired_no

df_no_sampled = df_no.orderBy(F.rand()).limit(desired_no)
df_yes_sampled = df_yes.orderBy(F.rand()).limit(desired_yes)

df_train = df_no_sampled.union(df_yes_sampled)
df_train.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("train_silver")

In [0]:
display(df_train)